<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
from __future__ import unicode_literals
from youtube_search import YoutubeSearch
import youtube_dl
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr
import bs4 as bs
import urllib.request
import re
import nltk
import heapq
import pubchempy as pcp
import smtplib,ssl
import requests
import json
import pandas as pd
import time
import yagmail        
from wit import Wit


chemicals=[]

access_token_wit='FE5AVTCKI4WL7X2S4RCZPS4L7D53S5QP'
#question='Can I combine the benzene and hexanol?'
question=input('Please ask a question about the chemical compound that you want to know about:')
def wit_request(question,access_token_wit):
    
    client = Wit(access_token=access_token_wit)
    resp=client.message(msg=question)

    data_extraction=json.dumps(resp)
    data=json.loads(data_extraction)


    def depth(data):
        if "entities" in data:
            return 1 +  max([0] + list(map(depth, data['entities'])))      
        else:
            return 1
    levels_with_entities=depth(data)


    #print(json.dumps(data,indent=2,sort_keys=True))
    
    def json_extract(obj,key):
        arr=[]
        def extract(obj,arr,key):
            if isinstance(obj,dict):
                for k,v in obj.items():
                    if isinstance(v,(dict,list)):
                        extract(v,arr,key)
                    elif v==key:
                        if obj not in arr:
                            arr.append(obj)
            elif isinstance(obj,list):
                for item in obj:
                    extract(item,arr,key)
            return (arr)
        values=extract(obj,arr,key)
        return values
    #get intents
    intent=resp['intents'][0]['name']
    #extract chemicals that wit.ai found
    result_confirm=json_extract(data,'chemical_substance')
    chemicals=[]
    number_chemicals=len(result_confirm)
    for q in range(number_chemicals):
        chemicals.append(result_confirm[q]['value'])   
    
    #print(json.dumps(result_confirm,indent=2,sort_keys=True))
    #print('result confirm:',chemicals,intent)#result_confirm)
    
    return (chemicals,intent)
wit_results=wit_request(question,access_token_wit)
print('Compuesto Quimico: ',wit_results[0])
print('Intensión Encontrada (Intent):',wit_results[1])

Please ask a question about the chemical compound that you want to know about:Can I combine the benzene and hexanol?
Compuesto Quimico:  ['Benzene', 'Hexanol']
Intensión Encontrada (Intent): get_ghs_classification
